<h1 align = "center">PSC INNOVATIVE ASSIGNMENT</h1>
<h2 align = "center">AUTOMATIC MUSIC GENERATION</h2>
<h2 align = "center">21BCE002, 21BCE003, 21BCE020</h2>

<h3>LIBRARIES USED:</h3>

1.) music21

2.) glob

3.) os

4.) tqdm

5.) numpy

6.) random

7.) tensorflow - for the main audio processing using LSTM.

8.) keras(layers and models)

9.) sklearn - the machine learning library for training and testing the model.


CODE SNIPPET:
--------------------------------------------------------------------------------------------------------------------------------
<i>from music21 import *

<i>import glob

<i>from tqdm import tqdm

<i>import numpy as np

<i>import random

<i>from tensorflow.keras.layers import LSTM,Dense,Input,Dropout

<i>from tensorflow.keras.models import Sequential,Model,load_model

<i>from sklearn.model_selection import train_test_split

<h3>THE OBJECTIVE:</h3>

- Our goal is to create our own original piece of music by using various audio files.
- We achieved this by feeding a machine learning model some audio files in the form of 'midi' files.
- The model would read and analyse those files and will train itself to produce music in a way it understands.
- The 'end-product' of running this program will provide the user with a new 'midi' file which will be the program's own      composition of music

<h3>HOW THE MODEL 'READS AND ANALYSES':</h3>

- The machine learning model is fed various 'midi' files.
- It 'learns' from those files what music should sound like.
- By 'READING AND ANALYZING', we mean that the program takes all the midi files, separates all the files into various unique notes and chords the files are composed of.
- The unique chords and notes are then used for testing and training the model, which helps it identify what sounds like 'good music'.

<h4>USEFUL INFORMATION:</h4>

- A 'chord' is made up of multiple notes. Thus, it makes sense for the model to split the chords encountered in the midi files into its respective 'unique' notes for work.


CODE SNIPPET:
--------------------------------------------------------------------------------------------------------------------------------------
<i>def read_files(file):
    
<i> notes=[]

<i> notes_to_parse=None

<i> #parse the midi file

<i> midi=converter.parse(file)
 
<i> #seperate all instruments from the file
 
<i> instrmt=instrument.partitionByInstrument(midi)

<i>for part in instrmt.parts:

<i> #fetch data only of Piano instrument
 
<i> if 'Piano' in str(part):
 
<i> notes_to_parse=part.recurse()
 

<i> #iterate over all the parts of sub stream elements
 
<i> #check if element's type is Note or chord
 
<i> #if it is chord split them into notes
 
<i> for element in notes_to_parse:
 
<i> if type(element)==note.Note:
 
<i>  notes.append(str(element.pitch))
  
<i> elif type(element)==chord.Chord:
 
<i>  notes.append('.'.join(str(n) for n in element.normalOrder))

<i>#return the list of notes

<i>return notes

<i>#retrieve paths recursively from inside the directories/files

<i>file_path=[FILE_PATH]

<i>all_files=glob.glob('All Midi Files/'+file_path[0]+'/*.mid',recursive=True)

<i>#reading each midi file

<i>notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)])

<h3>BUILDING THE MODEL:</h3>

- We used LSTM to build the model.
- LSTM is a variety of RNNs(Recurrent Neural Networks), which is used by the model to learn long-term dependencies, especially in sequence prediction.

<h4>HOW DOES SEQUENCE PREDICTION COME INTO PLAY HERE?</h4>

- Sequence Prediction is used by the model to decide which note/chord is played after which note/chord, which is very important for generation of good music.


CODE SNIPPET:
--------------------------------------------------------------------------------------------------------------------------------------

<i>#create the model

<i>model = Sequential()

<i>#create two stacked LSTM layer with the latent dimension of 256

<i>model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))

<i>model.add(Dropout(0.2))

<i>model.add(LSTM(256))

<i>model.add(Dropout(0.2))

<i>model.add(Dense(256,activation='relu'))

<i>#fully connected layer for the output with softmax activation

<i>model.add(Dense(len(note2ind),activation='softmax'))

<i>model.summary()

<h3>TRAINING THE MODEL:</h3>

- The model is now trained using the training set of notes.
- We used 10 epochs to train the model.

<b>*EPOCHS: Epochs are the number of times the training set of data is passed into the model to train.</b>

SNAPSHOT OF THE CODE:
--------------------------------------------------------------------------------------------------------------------------------------

<i>#compile the model using Adam optimizer

<i>model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

<i>#train the model on training sets and validate on testing sets

<i>model.fit(

<i> x_train, y_train,
 
<i> batch_size = BatchSize,epochs = number_of_epochs,
 
<i> validation_data = (x_test,y_test))

<h3>SAMPLING PHASE:</h3>

- This is the section where our efforts bear fruit. This is where the program is able to compose music.
- The trained model is used to predict the position of the notes to produce good music.
- The notes taken for training in a random order, and the predicted notes from the dataset will be stored in another list.

CODE SNIPPET:
--------------------------------------------------------------------------------------------------------------------------------------

<i>#load the model

<i>model = load_model(“s2s”)

<i>#generate random index

<i>index = np.random.randint(0,len(x_test)-1)

<i>#get the data of generated index from x_test

<i>music_pattern = x_test[index]

<i>out_pred=[] #it will store predicted notes

<i>#iterate till 200 note is generated

<i>for i in range(200):

<i> #reshape the music pattern
 
<i> music_pattern = music_pattern.reshape(1,len(music_pattern),1)

<i> #get the maximum probability value from the predicted output
 
<i> pred_index = np.argmax(model.predict(music_pattern))
 
<i> #get the note using predicted index and
 
<i> #append to the output prediction list
 
<i> out_pred.append(ind2note[pred_index])
 
<i> music_pattern = np.append(music_pattern,pred_index)

<i> #update the music pattern with one timestep ahead
 
<i> music_pattern = music_pattern[1:]

<h3>SAVING THE FILE INTO A 'MIDI' FILE:</h3>

- The output notes is then saved into a 'midi' file, which can then be converted to a playable format and played on a media player.

CODE SNIPPET:
--------------------------------------------------------------------------------------------------------------------------------------

<i>output_notes = []

<i>for offset,pattern in enumerate(out_pred):

<i>#if pattern is a chord instance

<i>if ('.' in pattern) or pattern.isdigit():

<i> #split notes from the chord
 
<i> notes_in_chord = pattern.split('.')
 
<i> notes = []
 
<i> for current_note in notes_in_chord:
 
<i>  i_curr_note=int(current_note)
  
<i>  #cast the current note to Note object and
  
<i>  #append the current note
  
<i>  new_note = note.Note(i_curr_note)
  
<i>  new_note.storedInstrument = instrument.Piano()
  
 <i> notes.append(new_note)

<i> #cast the current note to Chord object
 
<i> #offset will be 1 step ahead from the previous note
 
<i> #as it will prevent notes to stack up
 
<i> new_chord = chord.Chord(notes)
 
<i> new_chord.offset = offset
 
<i> output_notes.append(new_chord)

<i>else:

 <i>#cast the pattern to Note object apply the offset and
 
 <i>#append the note
 
<i> new_note = note.Note(pattern)
 
<i> new_note.offset = offset
 
<i> new_note.storedInstrument = instrument.Piano()
 
<i> output_notes.append(new_note)

<i>#save the midi file

<i>midi_stream = stream.Stream(output_notes)

<i>midi_stream.write('midi', fp='pred_music.mid')